<img src = "logo-ucsp.png" alt="logo-ucsp" height="250" width="250" align = "right">  
**<font color=blue> MAESTRIA EN CIENCIAS DE LA COMPUTACIÓN </font>**  
**<font color=blue> CURSO DE SISTEMAS INTELIGENTES </font>**  
**<font color=blue> PROFESOR: DR. JOSE OCHOA LUNA </font>**  
<font color=blue> ALUMNA : ROXANA SOTO BARRERA </font>  
____________________________________________________________________________________________

# HMM to Named Entity Recognition en Español

In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pandas as pd
import numpy as np
from helper import accuracy, entity_count
#import pycrfsuite

print(sklearn.__version__)

0.20.0


## 1. Load Data


In [2]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [3]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_a_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
test_b_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))


CPU times: user 1.66 s, sys: 57.9 ms, total: 1.72 s
Wall time: 1.86 s


In [4]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

There are 6 tags:  
    PER: Person  
    ORG: Organization
    LOC: location  
    MISC: Miscellaneous (this represents things like products and nationalities)  
    O: Not an entity
    

In [5]:
print(len(train_sents))
print(len(test_a_sents))
print(len(test_b_sents))

8323
1915
1517


In [6]:
N_sentences =len(train_sents)

## 2. Preprocesing

Defining functions to preprocesing of dataset

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features



Now, I am obtaing the tags of data

In [8]:
def sent2labels(sent):
    return [label for token, postag, label in sent]


them, tokenizar cadena of text

In [9]:
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [10]:
sent2labels(train_sents[0])

['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']

In [11]:
sent2tokens(train_sents[2])

['El',
 'Abogado',
 'General',
 'del',
 'Estado',
 ',',
 'Daryl',
 'Williams',
 ',',
 'subrayó',
 'hoy',
 'la',
 'necesidad',
 'de',
 'tomar',
 'medidas',
 'para',
 'proteger',
 'al',
 'sistema',
 'judicial',
 'australiano',
 'frente',
 'a',
 'una',
 'página',
 'de',
 'internet',
 'que',
 'imposibilita',
 'el',
 'cumplimiento',
 'de',
 'los',
 'principios',
 'básicos',
 'de',
 'la',
 'Ley',
 '.']

Now, I defining the count that use the tag in the dataset

In [12]:
def find_states_counts(n_tag_sentences):
    tags = []
    #concatenate all tags
    for i in range(n_tag_sentences):
        tags = tags + sent2labels(train_sents[i])

    #{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}
    dic = dict()
    states = set(tags) #set
    states = dic.fromkeys(states,0)
    
    #count frecuency by tag
    for f in range(n_tag_sentences):
        tags = sent2labels(train_sents[f])
        for t in tags:
            states[t] = 1 + states[t]
        
    return states

In [13]:
states = find_states_counts(N_sentences)                            
N_STATES = len(states)
states

{'I-LOC': 1891,
 'B-PER': 4321,
 'I-PER': 3903,
 'I-ORG': 4992,
 'B-ORG': 7390,
 'B-MISC': 2173,
 'I-MISC': 3212,
 'O': 231920,
 'B-LOC': 4913}

Calculating probabilities 

In [14]:
def calculate_start_prob(dic, total_sentences):
    start_probabilities = {}
    start_probabilities = start_probabilities.fromkeys(dic.keys(),0)
    for i in range(N_sentences):
        tag = sent2labels(train_sents[i])[0]
        start_probabilities[tag] = 1+ start_probabilities[tag]
    
    total = sum(start_probabilities.values())
    print(total)
    
    assert total == total_sentences, "exist a problem, not equal sum..."
    
    for i in start_probabilities:
        start_probabilities[i] = float(start_probabilities[i]/total)
    
    assert 1.0 == sum(start_probabilities.values()), "exist a problem, not equal to 1.0..."
    
    return start_probabilities

In [15]:
initial_probabilities = calculate_start_prob(states,N_sentences)
initial_probabilities

8323


{'I-LOC': 0.00012014898474107894,
 'B-PER': 0.07232968881412952,
 'I-PER': 0.0,
 'I-ORG': 0.0,
 'B-ORG': 0.03556409948335937,
 'B-MISC': 0.016099963955304577,
 'I-MISC': 0.0,
 'O': 0.7998317914213625,
 'B-LOC': 0.07605430734110297}

### Matrix of Transition

In [16]:
#Matriz de Transicion
def matrix_transitions(states, n_states, n_sentences):
    data = np.zeros([n_states, n_states], dtype = float)
    transitions = pd.DataFrame(data, index= states.keys(),columns=states.keys())
#    print('initial transitions: ',transitions)
    for s in range(n_sentences):
        tags = sent2labels(train_sents[s])
        for j in range(len(tags)-1):
            if tags[j]!=tags[j+1]:
                transitions.at[tags[j],tags[j+1]] += 1
                #print('->',tags[j],tags[j+1])
    
 #   print('sums transitions: ',transitions)
    for idx in transitions.index:
        for  col in transitions.columns:
            transitions[col][idx] /=  states[idx]    
    return transitions

In [17]:
transitions = matrix_transitions(states, N_STATES,N_sentences)
transitions

,I-LOC,B-PER,I-PER,I-ORG,B-ORG,B-MISC,I-MISC,O,B-LOC
I-LOC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.563194,0.001058
B-PER,0.000000,0.000000,0.665587,0.000000,0.000231,0.000000,0.000000,0.333256,0.000694
I-PER,0.000000,0.001537,0.000000,0.000000,0.000000,0.000000,0.000000,0.729439,0.005893
I-ORG,0.000000,0.004006,0.000000,0.000000,0.000601,0.000200,0.000000,0.443910,0.000401
B-ORG,0.000000,0.005954,0.000000,0.303383,0.000000,0.000135,0.000000,0.686333,0.004060
B-MISC,0.000000,0.003221,0.000000,0.000000,0.004602,0.000000,0.577543,0.411873,0.002761
I-MISC,0.000000,0.002491,0.000000,0.000000,0.000623,0.002179,0.000000,0.383562,0.000934
O,0.000000,0.015609,0.000000,0.000000,0.030515,0.008753,0.000000,0.000000,0.018045
B-LOC,0.216975,0.002646,0.000000,0.000000,0.000000,0.000000,0.000000,0.774883,0.000000


### Matrix of emision

In [18]:
len(sent2tokens(train_sents[2]))
all_sentences = []
for s in range(N_sentences):
    all_sentences += sent2tokens(train_sents[s])
    
len(all_sentences)
all_sentences = set(all_sentences) #set
all_sentences = list(all_sentences)
N_WORDS = len(all_sentences)

data = np.zeros([N_STATES, N_WORDS], dtype = float)
emissions = pd.DataFrame(data, index= states.keys(),columns=all_sentences)

def calculated_emissions():
    for index in range(N_sentences):
        sentence = sent2tokens(train_sents[index])
        tags = sent2labels(train_sents[index])

        for j in range(len(tags)):
            emissions[sentence[j]][tags[j]] += 1
    
    for idx in emissions.index:
        for col in emissions.columns:
            if states[idx] != 0:
                emissions[col][idx] /= states[idx]
            #print(emissions[sentence[j]][tags[j]])

In [19]:
calculated_emissions()
emissions


,siguieron,recorre,israelí,Venancio,ineludible,iguanodontes,reunirse,"7,70",Secretaría,coyuntural,...,ratificaron,Lobato,13.60,querían,21.53,costo,coaccionar,mío,513,demandada
I-LOC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B-PER,0.000000,0.000000,0.000000,0.000231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-PER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-ORG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B-ORG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000406,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B-MISC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-MISC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
O,0.000004,0.000004,0.000289,0.000000,0.000013,0.000004,0.000013,0.000004,0.000000,0.000004,...,0.000009,0.000000,0.000013,0.000004,0.000009,0.000017,0.000009,0.000009,0.000004,0.000004
B-LOC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000204,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
def transition(ei_row,ef_col):
    return transitions[ef_col][ei_row]

def emission(ei_row,ef_col):
    return emissions[ef_col][ei_row]

def max_p(f1,f2):
    maximo = f1 if f1 >= f2 else f2
    return maximo


In [21]:
transition('I-LOC','O')

0.5631940772078265

### Viterbi

In [25]:
def algoritm_viterbi(sequence):
    tags_rpta = []
    for st in states:
        dv_table.at[st,0] = initial_probabilities[st] + emission(st,sequence[0]) 
        for i in range(1,len(sequence),1):
            for st in states:
                #probabilidades del estado anterior
                col_dv_table = dv_table.loc[:,i-1] #probabilidades P(t-1)
                col_dv_table = np.array(col_dv_table)

                 #probabilidades de transicion S(t-1)->S(t)
                row_transition = transitions.loc[st,:]
                row_transition = np.array(row_transition)

                m = col_dv_table + row_transition
                p = emission(st,sequence[i]) + np.max(m)
                dv_table.at[st,i] = p
    
    for i in range(len(sequence)):
        col = dv_table.loc[dv_table[i].idxmax()]
        tags_rpta.append(col.name)
    return tags_rpta

## 3. Test

In [30]:
idx=2
sequence = ['El', 'Abogado', 'General']
l_seq = len(sequence)
data = np.zeros([N_STATES,l_seq])
dv_table = pd.DataFrame(data,index=states,columns=np.arange(l_seq))


In [31]:
tags = algoritm_viterbi(sequence)
tags
dv_table


,0,1,2
I-LOC,0.000120,1.368493,1.581769
B-PER,0.073255,1.139250,2.200325
I-PER,0.000512,1.534738,1.587356
I-ORG,0.002204,1.249209,1.584790
B-ORG,0.036782,1.491632,1.585054
B-MISC,0.020702,1.217172,1.766403
I-MISC,0.002491,1.188861,1.583607
O,0.805299,0.805299,1.598227
B-LOC,0.080329,1.580182,1.585469


## 4. Conclusion

4.1 The data set conll2002 (esp) of nltk is used  
4.2 To this end, the data set is included, the calculation of the parameters of the model by means of frequency counting  
4.3 I used the Viterbi algorithm for the decoding of sentences is implemented.

## 5. References


[1] https://github.com/oucler/HMM-Tagger/blob/master/HMM%20Tagger.ipynb  
[2] https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb  
[3] https://github.com/udacity/hmm-tagger/blob/master/HMM%20Tagger.ipynb   
[4] https://github.com/lopatovsky/HMMs/blob/master/hmms.ipynb    
[5] https://github.com/hankcs/hidden-markov-model/blob/master/test.py   
[6] https://github.com/oucler/HMM-Tagger   
[7] https://github.com/oucler/HMM-Tagger/blob/master/HMM%20Tagger.ipynb   
[8] https://github.com/udacity/hmm-tagger/blob/master/HMM%20warmup%20(optional).ipynb   
[9] https://github.com/thjashin/hmm/blob/master/hmm.ipynb  
